# Assignment2 - Supervised Learning flow

# Part 1(a) Student details:
* Please write the First-Name, First letter of Last-Name and last 4 digits of the i.d. for each student. 

In [ ]:
# stundent details example: John S. 9812
#                       student details 1: Ilay W. 0725
# (if exists)           student details 2: Aharoni C. 1047
# (if exists)           student details 3: Adir B.
# (if exists&premitted) student details 4: 

## Part 1(b) - Chat-GPT/other AI-agent/other assistance used:
* If you changed the prompt until you got a satisfying answer, please add all versions
* don't delete "pre" tags, so new-line is supported
* double click the following markdown cell to change
* press shift+enter to view
* Add information:

#### Add information in this Markdown cell (double click to change, shift-enter to view)
<pre>   
AI agent name:
Goal:
Propmpt1:
    
Propmpt2:
    
Propmpt3: 


AI agent name 2:
Goal:
Propmpt1:
    
Propmpt2:
    
Propmpt3: 

Other assistanse:    
</pre>

## Part 1(c) - Learning Problem and dataset explaination.
* Please explain in one paragraph
* don't delete "pre" tags, so new-line is supported
* double click the following markdown cell to change
* press shift+enter to view
* Add explaining text:

#### Add information in this Markdown cell (double click to change, shift-enter to view)
<pre>




    
</pre>

## Part 2 - Initial Preparations 
You could add as many code cells as needed

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer

In [ ]:
train_df = pd.read_csv('titanic_train.csv')
test_df = pd.read_csv('titanic_test.csv')

print('First 5 training rows:')
print(train_df.head())
print('\nFirst 5 test rows:')
print(test_df.head())

In [ ]:
sns.set(style='whitegrid')

# Survival counts by passenger class
plt.figure(figsize=(6, 4))
sns.countplot(data=train_df, x='Pclass', hue='Survived')
plt.title('Survival count by Passenger Class')
plt.savefig('eda_pclass.png')
plt.close()

# Age distribution for survived vs. not survived passengers
plt.figure(figsize=(6, 4))
sns.histplot(data=train_df, x='Age', hue='Survived', kde=True)
plt.title('Age distribution by survival')
plt.savefig('eda_age.png')
plt.close()

# Correlation heatmap between the numeric features
plt.figure(figsize=(6, 4))
cor = train_df.drop('Survived', axis=1).corr()
sns.heatmap(cor, annot=True, cmap='coolwarm')
plt.title('Feature correlation')
plt.savefig('eda_corr.png')
plt.close()

## Part 3 - Experiments
You could add as many code cells as needed

In [ ]:
X_train = train_df.drop('Survived', axis=1)
y_train = train_df['Survived']

param_grid = [
    {
        'clf': [LogisticRegression(max_iter=1000)],
        'clf__C': [0.1, 1.0, 10.0]
    },
    {
        'clf': [RandomForestClassifier(random_state=42)],
        'clf__n_estimators': [50, 100],
        'clf__max_depth': [5, 10, None]
    }
]

scorer = make_scorer(f1_score)

# Pipeline contains a standard scaler followed by the classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression())  # replaced by grid search
])

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=scorer,
    n_jobs=-1,
    refit=True
)

print('\nRunning grid search (this may take a moment)...')

grid.fit(X_train, y_train)

results = pd.DataFrame(grid.cv_results_)
print('\nGrid search results (mean test score):')
print(results[['params', 'mean_test_score', 'std_test_score']])

print('\nBest parameters:', grid.best_params_)
best_model = grid.best_estimator_

## Part 4 - Training 
Use the best combination of feature engineering, model (algorithm and hyperparameters) from the experiment part (part 3)

In [ ]:
best_model.fit(X_train, y_train)

## Part 5 - Apply on test and show model performance estimation

In [ ]:
X_test = test_df.drop('Survived', axis=1)
y_test = test_df['Survived']
y_pred = best_model.predict(X_test)

test_f1 = f1_score(y_test, y_pred)
print('\nTest F1 score:', test_f1)

print('\nFirst 5 predictions:')
print(pd.DataFrame({'Actual': y_test[:5].values,
                    'Predicted': y_pred[:5]}))